In [ ]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
baseline_takeup_rates = {
    "snap": 0.78,
    "medicaid": 0.88,
    "aca_ptc": 0.622
}


In [3]:
# Create baseline simulation
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")


In [4]:
# Store results
results = []


In [5]:
# Test each program with different takeup rates
for program, rates in takeup_scenarios.items():
    print(f"\n{'='*60}")
    print(f"{program.upper()} - Testing takeup rates: {rates}")
    print(f"{'='*60}")
    
    # Map program names to reform parameter names
    param_map = {
        "snap": "gov.usda.snap.takeup_rate",
        "medicaid": "gov.hhs.medicaid.takeup_rate",
        "aca_ptc": "gov.aca.takeup_rate"
    }
    
    # Calculate baseline
    baseline_values = baseline.calculate(program, map_to="person", period=2026)
    baseline_recipients = (baseline_values > 0).sum() / 1e6
    print(f"Baseline recipients: {baseline_recipients:.2f} million")
    
    # Test each takeup rate
    for rate in rates:
        # Create reform for this rate
        reform = Reform.from_dict(
            {
                param_map[program]: {
                    "2026-01-01.2100-12-31": rate
                }
            },
            country_id="us"
        )
        
        # Run simulation
        reformed_sim = Microsimulation(
            dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5", 
            reform=reform
        )
        reformed_values = reformed_sim.calculate(program, map_to="person", period=2026)
        
        # Calculate statistics
        reformed_recipients = (reformed_values > 0).sum() / 1e6
        people_gained = ((baseline_values == 0) & (reformed_values > 0)).sum() / 1e6
        people_lost = ((baseline_values > 0) & (reformed_values == 0)).sum() / 1e6
        net_change = reformed_recipients - baseline_recipients
        
        # Store and print results
        results.append({
            "program": program,
            "takeup_rate": rate,
            "baseline_millions": baseline_recipients,
            "reformed_millions": reformed_recipients,
            "gained_millions": people_gained,
            "lost_millions": people_lost,
            "net_change_millions": net_change
        })
        
        print(f"\nTakeup rate {rate:.0%}:")
        print(f"  Reformed recipients: {reformed_recipients:.2f} million")
        print(f"  Net change: {net_change:+.2f} million")



SNAP - Testing takeup rates: [0.7, 0.85, 1.0]
Baseline recipients: 63.88 million

Takeup rate 70%:
  Reformed recipients: 55.44 million
  Net change: -8.44 million

Takeup rate 85%:
  Reformed recipients: 67.17 million
  Net change: +3.29 million

Takeup rate 100%:
  Reformed recipients: 97.32 million
  Net change: +33.44 million

MEDICAID - Testing takeup rates: [0.6, 0.8, 0.95]
Baseline recipients: 89.72 million

Takeup rate 60%:
  Reformed recipients: 55.05 million
  Net change: -34.67 million

Takeup rate 80%:
  Reformed recipients: 75.30 million
  Net change: -14.42 million

Takeup rate 95%:
  Reformed recipients: 91.32 million
  Net change: +1.60 million

ACA_PTC - Testing takeup rates: [0.3, 0.5, 0.7]
Baseline recipients: 30.28 million

Takeup rate 30%:
  Reformed recipients: 13.52 million
  Net change: -16.76 million

Takeup rate 50%:
  Reformed recipients: 20.04 million
  Net change: -10.24 million

Takeup rate 70%:
  Reformed recipients: 31.38 million
  Net change: +1.10 mil

In [6]:
# Create summary DataFrame
summary_df = pd.DataFrame(results)
print("\n\nSUMMARY TABLE:")
print(summary_df.round(2))

# Pivot table for easier comparison
pivot_df = summary_df.pivot(index='takeup_rate', columns='program', values='net_change_millions')
print("\n\nNET CHANGE BY TAKEUP RATE (millions):")
print(pivot_df.round(2))



SUMMARY TABLE:
    program  takeup_rate  baseline_millions  reformed_millions  \
0      snap         0.70              63.88              55.44   
1      snap         0.85              63.88              67.17   
2      snap         1.00              63.88              97.32   
3  medicaid         0.60              89.72              55.05   
4  medicaid         0.80              89.72              75.30   
5  medicaid         0.95              89.72              91.32   
6   aca_ptc         0.30              30.28              13.52   
7   aca_ptc         0.50              30.28              20.04   
8   aca_ptc         0.70              30.28              31.38   

   gained_millions  lost_millions  net_change_millions  
0              0.0           0.00                -8.44  
1              0.0           0.00                 3.29  
2              0.0           0.00                33.44  
3              0.0           0.01               -34.67  
4              0.0           0.00    